### Генерация поэзии с помощью нейронных сетей: шаг 1
##### Автор: [Радослав Нейчев](https://www.linkedin.com/in/radoslav-neychev/), @neychev

Ваша основная задача: научиться генерироват стихи с помощью простой рекуррентной нейронной сети (Vanilla RNN). В качестве корпуса текстов для обучения будет выступать роман в стихах "Евгений Онегин" Александра Сергеевича Пушкина.

In [20]:
# do not change the code in the block below
# __________start of block__________
import string
import os
from random import sample

import numpy as np
import torch, torch.nn as nn
import torch.nn.functional as F

from IPython.display import clear_output

import matplotlib.pyplot as plt
# __________end of block__________

In [21]:
# do not change the code in the block below
# __________start of block__________
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print('{} device is available'.format(device))
# __________end of block__________

cpu device is available


#### 1. Загрузка данных.

In [22]:
# do not change the code in the block below
# __________start of block__________
!wget https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/onegin.txt
    
with open('onegin.txt', 'r') as iofile:
    text = iofile.readlines()
    
text = "".join([x.replace('\t\t', '').lower() for x in text])
# __________end of block__________

Python(43894) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


--2024-11-04 21:06:36--  https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/onegin.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 262521 (256K) [text/plain]
Saving to: ‘onegin.txt.1’

onegin.txt.1        100%[===================>] 256,37K  --.-KB/s    in 0,06s   

2024-11-04 21:06:37 (4,21 MB/s) - ‘onegin.txt.1’ saved [262521/262521]



#### 2. Построение словаря и предобработка текста
В данном задании требуется построить языковую модель на уровне символов. Приведем весь текст к нижнему регистру и построим словарь из всех символов в доступном корпусе текстов. Также добавим токен `<sos>`.

In [23]:
# do not change the code in the block below
# __________start of block__________
tokens = sorted(set(text.lower())) + ['<sos>']
num_tokens = len(tokens)

assert num_tokens == 84, "Check the tokenization process"

token_to_idx = {x: idx for idx, x in enumerate(tokens)}
idx_to_token = {idx: x for idx, x in enumerate(tokens)}

assert len(tokens) == len(token_to_idx), "Mapping should be unique"

print("Seems fine!")


text_encoded = [token_to_idx[x] for x in text]
# __________end of block__________

Seems fine!


__Ваша задача__: обучить классическую рекуррентную нейронную сеть (Vanilla RNN) предсказывать следующий символ на полученном корпусе текстов и сгенерировать последовательность длины 100 для фиксированной начальной фразы.

Вы можете воспользоваться кодом с занятие №6 или же обратиться к следующим ссылкам:
* Замечательная статья за авторством Andrej Karpathy об использовании RNN: [link](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
* Пример char-rnn от Andrej Karpathy: [github repo](https://github.com/karpathy/char-rnn)
* Замечательный пример генерации поэзии Шекспира: [github repo](https://github.com/spro/practical-pytorch/blob/master/char-rnn-generation/char-rnn-generation.ipynb)

Данное задание является достаточно творческим. Не страшно, если поначалу оно вызывает затруднения. Последняя ссылка в списке выше может быть особенно полезна в данном случае.

Далее для вашего удобства реализована функция, которая генерирует случайный батч размера `batch_size` из строк длиной `seq_length`. Вы можете использовать его при обучении модели.

In [24]:
# do not change the code in the block below
# __________start of block__________
batch_size = 256
seq_length = 100
start_column = np.zeros((batch_size, 1), dtype=int) + token_to_idx['<sos>']

def generate_chunk():
    global text_encoded, start_column, batch_size, seq_length

    start_index = np.random.randint(0, len(text_encoded) - batch_size*seq_length - 1)
    data = np.array(text_encoded[start_index:start_index + batch_size*seq_length]).reshape((batch_size, -1))
    yield np.hstack((start_column, data))
# __________end of block__________    

Пример батча:

In [25]:
next(generate_chunk())

array([[83, 59, 49, ...,  1, 56, 50],
       [83, 58, 62, ..., 61, 59, 57],
       [83,  5,  0, ..., 59, 12,  1],
       ...,
       [83,  1, 62, ...,  1, 62,  1],
       [83, 63, 61, ..., 66,  1, 59],
       [83, 58, 45, ...,  1, 55, 61]])

Далее вам предстоит написать код для обучения модели и генерации текста.

In [26]:
# Define the RNN model with Dropout
class CharRNN(nn.Module):
    def __init__(self, num_tokens, hidden_size, num_layers=1, dropout_prob=0.5):
        super(CharRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size=num_tokens, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout_prob)
        self.fc = nn.Linear(hidden_size, num_tokens)
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x, hidden):
        out, hidden = self.rnn(x, hidden)
        out = self.dropout(out)  # Apply dropout to the RNN output
        out = self.fc(out)
        return out, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)

# Initialize the model, loss function, and optimizer
hidden_size = 128
num_layers = 1
dropout_prob = 0.5  # Set dropout probability
model = CharRNN(num_tokens, hidden_size, num_layers, dropout_prob).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 8_192
for epoch in range(num_epochs):
    model.train()
    hidden = model.init_hidden(batch_size)
    for batch in generate_chunk():
        inputs = torch.tensor(batch[:, :-1], dtype=torch.int64).to(device)
        targets = torch.tensor(batch[:, 1:], dtype=torch.int64).to(device)

        # One-hot encode inputs
        inputs_one_hot = F.one_hot(inputs, num_classes=num_tokens).float()

        # Forward pass
        outputs, hidden = model(inputs_one_hot, hidden)
        loss = criterion(outputs.view(-1, num_tokens), targets.view(-1))

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

/Users/melodiz/code/ML_Trainings/env/lib/python3.12/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn(


Epoch [1/8192], Loss: 4.4317
Epoch [2/8192], Loss: 4.4040
Epoch [3/8192], Loss: 4.3778
Epoch [4/8192], Loss: 4.3487
Epoch [5/8192], Loss: 4.3151
Epoch [6/8192], Loss: 4.2746
Epoch [7/8192], Loss: 4.2231
Epoch [8/8192], Loss: 4.1432
Epoch [9/8192], Loss: 4.0416
Epoch [10/8192], Loss: 3.9098
Epoch [11/8192], Loss: 3.8083
Epoch [12/8192], Loss: 3.7018
Epoch [13/8192], Loss: 3.6460
Epoch [14/8192], Loss: 3.6098
Epoch [15/8192], Loss: 3.5579
Epoch [16/8192], Loss: 3.5543
Epoch [17/8192], Loss: 3.5191
Epoch [18/8192], Loss: 3.5065
Epoch [19/8192], Loss: 3.5024
Epoch [20/8192], Loss: 3.4766
Epoch [21/8192], Loss: 3.4826
Epoch [22/8192], Loss: 3.4659
Epoch [23/8192], Loss: 3.4713
Epoch [24/8192], Loss: 3.4386
Epoch [25/8192], Loss: 3.4532
Epoch [26/8192], Loss: 3.4374
Epoch [27/8192], Loss: 3.4454
Epoch [28/8192], Loss: 3.4405
Epoch [29/8192], Loss: 3.4351
Epoch [30/8192], Loss: 3.4421
Epoch [31/8192], Loss: 3.4198
Epoch [32/8192], Loss: 3.4267
Epoch [33/8192], Loss: 3.4143
Epoch [34/8192], Lo

В качестве иллюстрации ниже доступен график значений функции потерь, построенный в ходе обучения авторской сети (сам код для ее обучения вам и предстоит написать).

Шаблон функции `generate_sample` также доступен ниже. Вы можете как дозаполнить его, так и написать свою собственную функцию с нуля. Не забывайте, что все примеры в обучающей выборке начинались с токена `<sos>`.

In [27]:
# Function to generate text
def generate_sample(char_rnn, seed_phrase=None, max_length=200, temperature=1.0, device=device):
    if seed_phrase is not None:
        x_sequence = [token_to_idx['<sos>']] + [token_to_idx[token] for token in seed_phrase]
    else: 
        x_sequence = [token_to_idx['<sos>']]

    x_sequence = torch.tensor([x_sequence], dtype=torch.int64).to(device)
    hidden = char_rnn.init_hidden(1)

    generated_text = seed_phrase if seed_phrase else ''
    for _ in range(max_length - len(generated_text)):
        inputs_one_hot = F.one_hot(x_sequence, num_classes=num_tokens).float()
        outputs, hidden = char_rnn(inputs_one_hot, hidden)
        outputs = outputs[:, -1, :] / temperature
        probabilities = F.softmax(outputs, dim=-1).cpu().data.numpy()
        next_token = np.random.choice(num_tokens, p=probabilities.ravel())
        generated_text += idx_to_token[next_token]
        x_sequence = torch.cat((x_sequence, torch.tensor([[next_token]], dtype=torch.int64).to(device)), dim=1)

    return generated_text

In [28]:
# def generate_sample(char_rnn, seed_phrase=None, max_length=200, temperature=1.0, device=device):
#     '''
#     The function generates text given a phrase of length at least SEQ_LENGTH.
#     :param seed_phrase: prefix characters. The RNN is asked to continue the phrase
#     :param max_length: maximum output length, including seed_phrase
#     :param temperature: coefficient for sampling.  higher temperature produces more chaotic outputs,
#                         smaller temperature converges to the single most likely output
#     '''
    
#     if seed_phrase is not None:
#         x_sequence = [token_to_idx['<sos>']] + [token_to_idx[token] for token in seed_phrase]
#     else: 
#         x_sequence = [token_to_idx['<sos>']]

#     x_sequence = torch.tensor([x_sequence], dtype=torch.int64).to(device)
    
#     #feed the seed phrase, if any
            
#     # your code here
    
#     return ''.join([tokens[ix] for ix in x_sequence.cpu().data.numpy()[0]])

Пример текста сгенерированного обученной моделью доступен ниже. Не страшно, что в тексте много несуществующих слов. Используемая модель очень проста: это простая классическая RNN.

In [33]:
print(generate_sample(model, ' мой дядя самых честных правил', max_length=500, temperature=0.8))

 мой дядя самых честных правилсе

безутря с мозя сей калола
порож, нелетки наютрет.
но натизнок подлотел сердом
вои пручто они удрине,
неждо то тревин слови слилью;
и полв ин вовонехален.

прость у рож,
нимодане занитмый понвих;
пикой ял бо модрусть о сорейспот
таньяно в перевом иль?
думароко влюбенныль тот
лае в тасонец скалану дал
пальпою приспичты пизый на пиль
ело а коретраю тымой
стерня м поремнась пастимол;
вотвыынененю вострень
меполетванет про и врага

л
земил лашяль кока пошаю,
осчего с


### Сдача задания
Сгенерируйте десять последовательностей длиной 500, используя строку ' мой дядя самых честных правил'. Температуру для генерации выберите самостоятельно на основании визуального качества генериуремого текста. Не забудьте удалить все технические токены в случае их наличия.

Сгенерированную последовательность сохрание в переменную `generated_phrase` и сдайте сгенерированный ниже файл в контест.

In [30]:
seed_phrase = ' мой дядя самых честных правил'

In [31]:
generated_phrases = [
    generate_sample(model, seed_phrase, max_length=500, temperature=0.8).replace('<sos>', '')
    for _ in range(10)
]

In [32]:
# do not change the code in the block below
# __________start of block__________

import json
if 'generated_phrases' not in locals():
    raise ValueError("Please, save generated phrases to `generated_phrases` variable")

for phrase in generated_phrases:

    if not isinstance(phrase, str):
        raise ValueError("The generated phrase should be a string")

    if len(phrase) != 500:
        raise ValueError("The `generated_phrase` length should be equal to 500")

    assert all([x in set(tokens) for x in set(list(phrase))]), 'Unknown tokens detected, check your submission!'
    

submission_dict = {
    'token_to_idx': token_to_idx,
    'generated_phrases': generated_phrases
}

with open('submission_dict.json', 'w') as iofile:
    json.dump(submission_dict, iofile)
print('File saved to `submission_dict.json`')
# __________end of block__________

File saved to `submission_dict.json`


На этом задание завершено. Поздравляем!